The goal of this competition is to create algorithms and models that can solve tricky math problems written in LaTeX format. Your participation will help to advance AI models’ mathematical reasoning skills and drive frontier knowledge.

Description
Note: This is the second AIMO Progress Prize competition. It builds upon the first AIMO Progress Prize competition, which was won in July 2024 by Project Numina. This second competition has an increased prize pool, a new dataset of problems, increased compute for participants and updated rules for using open-source LLMs.

The ability to reason mathematically is a critical milestone for AI. Mathematical reasoning is the foundation for solving many complex problems, from engineering marvels to intricate financial models. However, current AI capabilities are limited in this area.

The AI Mathematical Olympiad (AIMO) Prize is a $10mn fund to spur the open development of AI models capable of performing as well as top human participants in the International Mathematical Olympiad (IMO).

This second AIMO Progress Prize competition has 110 math problems in algebra, combinatorics, geometry and number theory. The difficulty has been increased from the first competition, and the problems are now around the National Olympiad level. The problems have also been designed to be 'AI hard' in terms of the mathematical reasoning required, which was tested against current open LLMs' capabilities.

To address the challenge of train-test leakage, the competition uses novel math problems created by an international team of problem solvers. Using this transparent and fair evaluation framework, the competition will help to strengthen the benchmarks for assessing AI models' mathematical reasoning skills, without the risk of contamination from training data.
This latest AIMO Progress Prize competition offers an exciting opportunity to drive innovation in the field of AI for Math, while also fostering healthy competition and supporting open science.

Join us as we work towards a future where AI models’ mathematical reasoning skills are accurately and reliably assessed, driving progress and innovation.


Submissions are evaluated on the accuracy between their predicted labels and the ground-truth labels. In other words, submissions are ranked by the fraction of predicted labels that exactly match the ground-truth labels.

In this competition, every ground-truth label is an integer between 0 and 999, inclusive.

You should arrive at this number by taking the problem solution modulo 1000. If, for instance, you believe the solution to a problem is 65521 should be reported as 521 and -900 should be reported as 100. To be clear, for positive integers larger than 1000, this means: report the last three digits, discarding any initial zero(s). Thus 1009 should be reported as 9.

If a question asks for an answer a
 to be calculated modulo m
 where m
 is specified (not all questions are of this type), then calculate the residue a
 modulo m
 which is a′
 with 0≤a′<m
 and then report this answer modulo 1000
. For example, if asked to calculate the positive integer 2025
 modulo 999
, the final answer should be 27
. However, if asked to calculate 2025
 modulo 1013
, the final answer should be 12
.

Answers may require basic computations, e.g., ⌊1002–√⌋=141

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re

def clean_latex(text):
    """Clean and format LaTeX output for better readability."""
    # Remove LaTeX delimiters while preserving the math
    text = re.sub(r'\\\(|\\\)', '', text)
    
    # Format common math symbols
    replacements = {
        r'\sqrt': '√',
        r'\cdot': '×',
        r'\times': '×',
        r'\div': '÷',
        r'\le': '≤',
        r'\ge': '≥',
        r'\neq': '≠',
        r'\pm': '±',
    }
    
    for latex, symbol in replacements.items():
        text = text.replace(latex, symbol)
    
    # Clean up spaces around operators
    text = re.sub(r'\s*([+\-×÷=])\s*', r' \1 ', text)
    
    # Format exponents nicely
    text = re.sub(r'\^([0-9])', r'²', text)  # Simple case for squared
    text = re.sub(r'\^([0-9])', r'³', text)  # Simple case for cubed
    
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def generate_math_solution(prompt, max_length=512, temperature=0.7):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    huggingface_model = "Qwen/Qwen2.5-Math-1.5B-Instruct"
    
    tokenizer = AutoTokenizer.from_pretrained(huggingface_model)
    model = AutoModelForCausalLM.from_pretrained(huggingface_model)
    
    model.to(device)
    model.eval()
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            no_repeat_ngram_size=3,
            top_k=50,
            top_p=0.95,
        )
    
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Split the text into steps and format each step
    steps = output_text.split('\n')
    formatted_steps = []
    
    for step in steps:
        if step.strip():
            # Clean LaTeX in the step
            cleaned_step = clean_latex(step)
            formatted_steps.append(cleaned_step)
    
    # Join the steps with proper spacing
    return '\n\n'.join(formatted_steps)

def format_and_print_solution(solution):
    """Print the solution in a nicely formatted way."""
    print("\n=== Math Solution ===\n")
    
    # Split into lines and format each line
    lines = solution.split('\n')
    for line in lines:
        if line.strip():
            if line.startswith(('Step', '1.', '2.', '3.', '4.', '5.')):
                print(f"\n{line}")
            else:
                print(line)
    
    print("\n==================\n")

# Example usage
if __name__ == "__main__":
    prompt = "Solve the following equation: x² + 2x + 1 = 0"
    solution = generate_math_solution(prompt)
    format_and_print_solution(solution)